# 課題10 モデル評価と選択

配点
- Q1から3あわせて10P

In [48]:
import pandas as pd
import numpy as  np
import csv
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix,  accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot  as plt
%matplotlib inline

In [3]:
# Colaboratoryでは以下を実行して必要なファイルをダウンロード
!wget https://raw.githubusercontent.com/UTDataMining/2020A/master/1214/winequality-red.csv

/bin/sh: wget: command not found


## Q モデル評価と選択
UCI Machine Learning Repositoryに公開されているワインの品質データセットを用いて、ワインの理化学検査結果に基づく特徴量からワインの品質を予測することを考えます。

以下では赤ワインのデータセットを使用します。

[Wine Quality Data Set ](https://archive.ics.uci.edu/ml/datasets/wine+quality)

各ワインのデータは以下の11種類の特徴量からなります。
- 1 - fixed acidity 
- 2 - volatile acidity 
- 3 - citric acid 
- 4 - residual sugar 
- 5 - chlorides 
- 6 - free sulfur dioxide 
- 7 - total sulfur dioxide 
- 8 - density 
- 9 - pH 
- 10 - sulphates 
- 11 - alcohol 

また、各ワインには0から10の品質スコアが付与さています。11種類の特徴量を元にワインの品質を高品質（ワインのqualityが6以上）、低品質（ワインのqualityが6未満）の2値にして予測する2クラス分類問題を考えます。

In [49]:
df = pd.read_csv("winequality-red.csv", sep=";")

In [50]:
# 特徴量
X=df[ ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates','alcohol']].values

# 正解ラベルを1（高品質：ワインのqualityが6以上）か0（低品質：ワインのqualityが6未満）とする
y=df['quality'].values
y = (y>=6).astype(np.int) 
print(np.sum(y==1, axis=0)) # 高品質ワインデータの数
print(np.sum(y==0, axis=0)) # 低品質ワインデータの数

855
744


In [51]:
# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8, stratify=y) 

すべての特徴量を使ってロジスティック回帰によりワイン品質の分類をしてみます

[LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [52]:
# 標準化
sc = StandardScaler()
sc.fit(X_train)
norm_X_train =  sc.transform(X_train)
norm_X_test =  sc.transform(X_test)

# ロジスティック回帰モデル
lr=LogisticRegression(solver='liblinear',  multi_class='auto', C=1) 

lr.fit(norm_X_train, y_train)    
y_pred = lr.predict(norm_X_test)

print(confusion_matrix(y_test, y_pred)) # 混同行列
print(accuracy_score(y_test, y_pred)) # accuracy
print(precision_score(y_test, y_pred)) # precision
print(recall_score(y_test, y_pred)) # recall
print(f1_score(y_test, y_pred))  # f値

# 特徴量ごとのパラメータ重み
pd.DataFrame(lr.coef_, index=['weight'], columns=df.drop("quality", axis=1).columns.values)

[[107  42]
 [ 47 124]]
0.721875
0.7469879518072289
0.7251461988304093
0.7359050445103857


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
weight,0.228423,-0.614599,-0.30209,0.090223,-0.135772,0.19444,-0.545962,-0.118573,-0.049804,0.463159,0.862989


## Q1
先のモデルについて、訓練データを用いた**5分割**交差検証により**F値**を評価指標として最適な正則化項の係数を決定してください。

- 特徴量を標準化してから交差検証を行ってください。
- `LogisticRegression()`の`solver`には`liblinear`を使用してください。
- `LogisticRegression()`の引数`C`の値の逆数が正則化項の係数になることに注意してください。
- `C`は、 0.1, 1, 10, のいずれかとします。

## Q2
Q1のモデルについて、さらに多項式を用いた特徴量生成を考えます。訓練データを用いた**5分割**交差検証により**F値**を評価指標として最適な正則化項の係数と多項式の次数の組み合わせ決定してください。

- 多項式に用いて生成した特徴量を標準化してから交差検証を行ってください。
- `LogisticRegression()`の`solver`には`liblinear`を使用してください。
- `LogisticRegression()`の引数`C`の値の逆数が正則化項の係数になることに注意してください。
- `C`は、 0.1, 1, 10, のいずれかとします。
- 多項式の次数は、1, 2, 3, のいずれかとします。


## Q3
Q2の交差検証で決定した正則化項の係数と多項式の次数により訓練データからロジスティック回帰モデルを学習し、テストデータで評価を行ってください。評価指標にはprecision, recall, F値を用いてください。